In [2]:
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators import H2ORandomForestEstimator
import pandas as pd

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.13" 2021-10-19; OpenJDK Runtime Environment JBR-11.0.13.7-1751.21-jcef (build 11.0.13+7-b1751.21); OpenJDK 64-Bit Server VM JBR-11.0.13.7-1751.21-jcef (build 11.0.13+7-b1751.21, mixed mode)
  Starting server from /opt/conda/lib/python3.12/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmprz_iye3j
  JVM stdout: /tmp/tmprz_iye3j/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmprz_iye3j/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,12 days
H2O_cluster_name:,H2O_from_python_unknownUser_ddmfbm
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.930 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [6]:
imp = pd.read_csv("../data/Churn_treino.csv", sep=";")

imp.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0,1,1,1,10134888,1
1,608,Spain,Female,41,1,8380786,1,0,1,11254258,0
2,502,France,Female,42,8,1596608,3,1,0,11393157,1
3,699,France,Female,39,1,0,2,0,0,9382663,0
4,850,Spain,Female,43,2,12551082,1,1,1,790841,0


In [7]:
imp = h2o.H2OFrame(imp)

train, test = imp.split_frame(ratios=[0.7])

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [8]:
train["Exited"] = train["Exited"].asfactor()

test["Exited"] = test["Exited"].asfactor()

In [9]:
# Run AutoML for 180 seconds, just for quick demonstration purposes
# Use `max_models` to limit the number of models built
modelo_auto = H2OAutoML(max_runtime_secs=180, sort_metric="AUC")

modelo_auto.train(y="Exited", training_frame=train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),14/67
# GBM base models (used / total),8/34
# XGBoost base models (used / total),1/19
# DeepLearning base models (used / total),3/11
# DRF base models (used / total),2/2
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [10]:
modelo_forest = H2ORandomForestEstimator(nfolds=5)

modelo_forest.train(y="Exited", training_frame=train)

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ORandomForestEstimator : Distributed Random Forest
Model Key: DRF_model_python_1725987412445_7848


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    50                 50                          537878                 19           20           19.92         797           911           851.2

ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.10936582372553705
RMSE: 0.3307050403691136
LogLoss: 0.695788675922422
Mean Per-Class Error: 0.25044102356033865
AUC: 0.8368251394614075
AUCPR: 0.6514433142354475
Gini: 0.6736502789228149

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3809523823715391
       0     1     Error    Rate
-----  ----  ----  -------  ---------------
0      5031  537   0.0964   (537.0/5568.0)
1      565   832   0.4044   (565.0/1397.0)
Total  5596  1369  0.1582   (1102.0/6965.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.380952     0.601591  174
max f2                       0.186275     0.668751  271
max f0point5                 0.608696     0.665355  96
max accuracy                 0.543478     0.860445  115
max precision                0.904762     0.926829  19
max recall                   0            1         399
max specificity              1            0.998563  0
max absolute_mcc             0.452381     0.516045  146
max min_per_class_accuracy   0.202991     0.756621  260
max mean_per_class_accuracy  0.270474     0.762315  220
max tns                      1            5560      0
max fns                      1            1320      0
max fps                      0            5568      399
max tps                      0            1397      399
max tnr                      1            0.998563  0
max fnr                      1            0.944882  0
max fpr                      0            1         399
max tpr                      0            1         399

Gains/Lift Table: Avg response rate: 20.06 %, avg score: 20.85 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0122039                   1                  4.51644   4.51644            0.905882         1           0.905882                    1                   0.0551181       0.0551181                  351.644   351.644            0.0536813
2        0.0208184                   0.944444           4.57021   4.53869            0.916667         0.951054    0.910345                    0.979746            0.0393701       0.0944882                  357.021   353.869            0.0921534
3        0.0307251                   0.9                4.6244    4.56633            0.927536         0.921719    0.915888                    0.961037            0.0458125       0.140301                   362.44    356.633            0.137068
4        0.041206                    0.866667           4.37101   4.51665            0.876712         0.881484    0.905923                    0.940802            0.0458125       0.186113                   337.101   351.665            0.181264
5        0.0501077                   0.831373           4.18154   4.45712            0.83871          0.847673    0.893983                   

In [17]:
ranking = modelo_auto.leaderboard

ranking = ranking.as_data_frame(use_multi_thread=True)

ranking[:10]

/opt/conda/lib/python3.12/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


,model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
0,StackedEnsemble_AllModels_4_AutoML_1_20240910_...,0.867845,0.327616,0.708751,0.226910,0.314977,0.099210
1,StackedEnsemble_AllModels_3_AutoML_1_20240910_...,0.867728,0.327540,0.709119,0.234214,0.314950,0.099193
2,StackedEnsemble_BestOfFamily_4_AutoML_1_202409...,0.865913,0.329193,0.706875,0.228795,0.315746,0.099696
3,GBM_grid_1_AutoML_1_20240910_171632_model_14,0.865372,0.331146,0.705021,0.226121,0.316448,0.100139
4,GBM_grid_1_AutoML_1_20240910_171632_model_27,0.865283,0.334230,0.696301,0.237799,0.318386,0.101370
5,StackedEnsemble_AllModels_2_AutoML_1_20240910_...,0.864740,0.329517,0.707358,0.239846,0.315558,0.099577
6,GBM_grid_1_AutoML_1_20240910_171632_model_7,0.864527,0.330827,0.704104,0.239393,0.316300,0.100046
7,GBM_grid_1_AutoML_1_20240910_171632_model_20,0.864438,0.332451,0.704490,0.230583,0.316994,0.100485
8,StackedEnsemble_BestOfFamily_3_AutoML_1_202409...,0.864122,0.330029,0.705974,0.235383,0.315890,0.099787
9,StackedEnsemble_AllModels_1_AutoML_1_20240910_...,0.864060,0.331082,0.704630,0.240568,0.316345,0.100074


In [18]:
performance = modelo_forest.model_performance(test)

performance

ModelMetricsBinomial: drf
** Reported on test data. **

MSE: 0.11490799959118819
RMSE: 0.33898082481342245
LogLoss: 0.4949554507958357
Mean Per-Class Error: 0.27334127087682675
AUC: 0.8325061978079332
AUCPR: 0.6506713893373356
Gini: 0.6650123956158664

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.47799999952316286
       0     1    Error    Rate
-----  ----  ---  -------  --------------
0      2257  138  0.0576   (138.0/2395.0)
1      313   327  0.4891   (313.0/640.0)
Total  2570  465  0.1486   (451.0/3035.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.478        0.591855  67
max f2                       0.18         0.674157  184
max f0point5                 0.6          0.654639  43
max accuracy                 0.478        0.8514    67
max precision                1            1         0
max recall                   0            1         315
max specificity              1            1         0
max absolute_mcc             0.478        0.513396  67
max min_per_class_accuracy   0.204        0.754687  167
max mean_per_class_accuracy  0.21         0.761114  163
max tns                      1            2395      0
max fns                      1            635       0
max fps                      0            2395      315
max tps                      0            640       315
max tnr                      1            1         0
max fnr                      1            0.992188  0
max fpr                      0            1         315
max tpr                      0            1         315

Gains/Lift Table: Avg response rate: 21.09 %, avg score: 21.46 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0108731                   0.96               4.45478   4.45478            0.939394         0.971515     0.939394                    0.971515            0.0484375       0.0484375                  345.478   345.478            0.0476024
2        0.0243822                   0.92               4.51086   4.48585            0.95122          0.92935      0.945946                    0.948153            0.0609375       0.109375                   351.086   348.585            0.107705
3        0.030972                    0.9                3.79375   4.3386             0.8              0.900667     0.914894                    0.93805             0.025           0.134375                   279.375   333.86             0.131035
4        0.0408567                   0.86               3.95182   4.24502            0.833333         0.87         0.895161                    0.921586            0.0390625       0.173437                   295.182   324.502            0.16801
5        0.0523888                   0.82               4.06473   4.20534            0.857143         0.831238     0.886792                    0.901698            0.046875        0.220312                   306.473   320.534            0.212797
6        0.101812                    0.62               3.28792   3.75999            0.693333         0.716882     0.79288                     0.811982            0.1625          0.382812                   228.792   275.999            0.35609
7        0.152883                    0.48               2.47818   3.3318             0.522581         0.536518     0.702586                    0.719962            0.126562        0.509375                   147.818   233.18             0.451755
8        0.200988    

In [19]:
test_prever = pd.read_csv("../data/Churn_prever.csv", sep=";")

test_prever = h2o.H2OFrame(test_prever)

previsao = modelo_auto.leader.predict(test_prever)

previsao

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict,p0,p1
0,0.897816,0.102184
0,0.918527,0.0814731
1,0.00818999,0.99181
0,0.977784,0.0222161
0,0.947586,0.0524135
0,0.930649,0.0693505
0,0.746334,0.253666
0,0.827523,0.172477
0,0.928335,0.0716646
